## **Pre-processing of commonly used data files**

In [2]:
import sys; sys.path.append('/home/clair/WWA'); from wwa import *

### **Global elevation**

! wget http://research.jisao.washington.edu/data_sets/elevation/elev.0.25-deg.nc

ds = wrap_lon(xr.open_dataset("../90_data/elev.0.25-deg.nc", decode_times = False).data.squeeze(drop = True))
ds = ds.assign_attrs(long_name = "elevation", units = "m above sea level",                    
                     source = "http://research.jisao.washington.edu/data_sets/elevation/").rename("elev")
ds.to_netcdf("../90_data/elevation-0.25deg-global.nc")